<a href="https://colab.research.google.com/github/JaideepDabral/neural-style-transfer/blob/main/Style_transfer_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask streamlit pyngrok diffusers transformers accelerate torch pillow --quiet

In [2]:
%%writefile api.py
from flask import Flask, request, jsonify, send_file
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
import PIL
import io

app = Flask(__name__)


model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, safety_checker=None
)
pipe.to("cuda")
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)


def edit_image(prompt, image):
    try:

        print(f"Editing Image: {image.size}, mode: {image.mode}")


        output = pipe(
            prompt,
            image=image,
            num_inference_steps=10,
            image_guidance_scale=1.2
        ).images[0]

        print("✅ Image processing successful")
        return output
    except Exception as e:
        print(f"❌ Error in edit_image: {str(e)}")  # Debugging
        return None


@app.route("/edit", methods=["POST"])
def process_image():
    try:

        print(f"📥 Received files: {request.files.keys()}")

        prompt = request.form.get("prompt")
        file = request.files.get("image")

        if not file or not prompt:
            return jsonify({"error": "Missing prompt or image"}), 400


        print(f"🔍 Processing file: {file.filename}, Content-Type: {file.content_type}")


        image = PIL.Image.open(file.stream).convert("RGB")


        print(f"🖼️ Image format: {image.format}, size: {image.size}")


        output_image = edit_image(prompt, image)

        if output_image is None:
            return jsonify({"error": "Image processing failed"}), 500


        img_io = io.BytesIO()
        output_image.save(img_io, format="PNG")
        img_io.seek(0)

        print("✅ Returning processed image")
        return send_file(img_io, mimetype="image/png")

    except Exception as e:
        print(f"❌ Flask Error: {str(e)}")
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)




Overwriting api.py


In [3]:
!pip install pyngrok flask-ngrok --quiet

import subprocess
import time
from pyngrok import ngrok


process = subprocess.Popen(["python3", "api.py"])


time.sleep(5)


ngrok.set_auth_token("2ur*************")
public_url = ngrok.connect(8080)
print(f"API is live at: {public_url}")


API is live at: NgrokTunnel: "https://554d-35-187-253-172.ngrok-free.app" -> "http://localhost:8080"


In [4]:
%%writefile app.py
import streamlit as st
import requests
import io
from PIL import Image


FLASK_API_URL = "https://554d-35-187-253-172.ngrok-free.app/"

st.title("NEURAL STYLE TRANSFER ")

uploaded_file = st.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])

prompt = st.text_input("Enter your instruction ")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", width=300)

    if st.button("Generate Edited Image"):
        with st.spinner("Processing..."):
            img_bytes = io.BytesIO()
            image.save(img_bytes, format="PNG")
            img_bytes.seek(0)

            files = {"image": img_bytes.getvalue()}
            data = {"prompt": prompt}

            response = requests.post(FLASK_API_URL, files={"image": img_bytes}, data=data)

            if response.status_code == 200:
                output_image = Image.open(io.BytesIO(response.content))
                st.image(output_image, caption="Edited Image", use_column_width=True)

                buf = io.BytesIO()
                output_image.save(buf, format="PNG")
                st.download_button("Download Image", buf.getvalue(), file_name="edited_image.png", mime="image/png")
            else:
                st.error("Error processing image. Try again.")


Overwriting app.py


In [5]:
!pip install streamlit pyngrok --quiet

import os
import time
import threading
from pyngrok import ngrok


def run_streamlit():
    os.system("streamlit run app.py &")


thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()


time.sleep(10)

public_url = ngrok.connect(8501)
print(f"Streamlit is live at: {public_url}")




Streamlit is live at: NgrokTunnel: "https://e852-35-187-253-172.ngrok-free.app" -> "http://localhost:8501"


In [6]:
!python -u app.py


2025-03-27 15:05:25.270 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 15:05:25.330 
  command:

    streamlit run app.py [ARGUMENTS]
2025-03-27 15:05:25.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 15:05:25.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 15:05:25.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 15:05:25.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 15:05:25.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-27 15:05:25.331 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when r